In [52]:
# Conexão com SQL Server
import pandas as pd
import pyodbc

server = 'WILLIAN\SQLEXPRESS'
database = 'SQL_DB_1'

conexaoDB = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                         f'SERVER={server};'
                         f'DATABASE={database};'
                         'Trusted_Connection=yes;')
cursor = conexaoDB.cursor()

In [53]:
#Suprimir avisos
import warnings
warnings.filterwarnings("ignore", category=UserWarning) # Esta parte de código é para suprimir avisos de UserWarming

Categoria = pd.read_sql('SELECT * FROM Categoria', conexaoDB)
Categoria.head(10)

,ID,Categoria
0,0,Moda e Acessórios
1,1,Cosméticos e Perfumaria
2,2,Eletrodomésticos
3,3,Livros
4,4,Celulares
5,5,Informática
6,6,Casa e Decoração
7,7,Eletrônicos
8,8,Esporte e Lazer
9,9,Brinquedos e Games


In [54]:
Query = 'SELECT * FROM Categoria'
df = pd.read_sql(Query,conexaoDB)
df.head(10)

,ID,Categoria
0,0,Moda e Acessórios
1,1,Cosméticos e Perfumaria
2,2,Eletrodomésticos
3,3,Livros
4,4,Celulares
5,5,Informática
6,6,Casa e Decoração
7,7,Eletrônicos
8,8,Esporte e Lazer
9,9,Brinquedos e Games


In [55]:
Query_clientes = """
    SELECT * FROM clientes AS CLI
    WHERE 1=1
    AND CLI.STATE IN('Acre','Mato Grosso','Goiás','Maranhão','Rondônia')
    AND CLI.FIRST_NAME IN('Stephanie','Laura','Heitor','Bruno')
"""
ClientesEspecificos = pd.read_sql(Query_clientes, conexaoDB)
ClientesEspecificos.head(10)

,id,created_at,first_name,last_name,email,cell_phone,country,state,street,number,additionals
0,15,2018-11-15 18:00:53,Stephanie,Barbosa,stephanie@usuario.com,9 1446-3560,Brasil,Mato Grosso,Sem Registros,Sem Registros,Conjunto 4
1,16,2017-12-12 21:29:54,Laura,Santos,laura@exemplo.com,9 7802-6152,Brasil,Maranhão,Rua 57 do Estado Maranhão,213.0,Sem Registros
2,22,2018-04-18 22:43:29,Heitor,Pereira,Sem Registros,9 4839-8118,Brasil,Maranhão,Avenida 84 do Estado Maranhão,948.0,Sem Registros
3,52,2017-12-04 17:42:34,Bruno,Anjos,bruno@teste.com,9 3004-3740,Brasil,Goiás,Rua 53 do Estado Goiás,825.0,Sem Registros
4,57,2018-08-23 03:31:05,Stephanie,Barboza,stephanie@meu_email.com,9 1168-6722,Brasil,Maranhão,Avenida 25 do Estado Maranhão,795.0,Sem Registros
5,66,2018-10-18 04:31:59,Stephanie,Abreu,stephanie@meu_email.com,9 6466-2389,Brasil,Mato Grosso,Rua 19 do Estado Mato Grosso,53.0,Conjunto 13


In [56]:
Query_Orden = """
 SELECT * FROM Ordens
"""
Ordem = pd.read_sql(Query_Orden, conexaoDB)
Ordem.head(10)

,id,created_at,customer_id,status
0,0,2019-08-28 03:36:31,91,entregue
1,1,2019-08-16 20:21:30,7,entregue
2,2,2019-07-11 11:00:06,26,pagamento_pendente
3,3,2019-08-27 21:04:41,92,pagamento_pendente
4,4,2019-11-03 19:58:37,63,entrega_pendente
5,5,2019-04-04 07:31:10,3,entrega_pendente
6,6,2019-05-26 07:22:06,42,entrega_pendente
7,7,2020-02-07 21:48:37,51,pagamento_pendente
8,8,2020-01-09 11:26:29,35,carrinho
9,9,2019-04-27 17:34:19,5,pagamento_pendente


In [57]:
Query_all = """

WITH QUERY AS(
    SELECT 
      T1.ORDER_ID
    , T1.TOTAL_PRICE
    , T2.CREATED_AT
    , T2.STATUS
    , T3.Nome       AS PRODUTO
    , T3.Price
    , T4.Categoria
    , T5.FIRST_NAME AS NomeCliente
    , T5.email
    , T5.cell_phone
    , T5.state      AS Estado
	, GETDATE()     AS Dh
	, CAST(GETDATE() AS DATE) AS DataHoje
	, CAST(GETDATE() AS TIME) AS Hora
	, CASE WHEN  DATEDIFF(DAY, CAST(T2.CREATED_AT AS DATE), CAST(GETDATE() AS DATE)) IS NULL THEN 0
	       ELSE  DATEDIFF(DAY, CAST(T2.CREATED_AT AS DATE), CAST(GETDATE() AS DATE)) END  AS DiasDesdeCriacao
    , CASE WHEN  DATEDIFF(MONTH, CAST(T2.CREATED_AT AS DATE), CAST(GETDATE() AS DATE)) IS NULL THEN 0
	       ELSE  DATEDIFF(MONTH, CAST(T2.CREATED_AT AS DATE), CAST(GETDATE() AS DATE)) END AS MesesDesdeCriacao
	, CASE WHEN  DATEDIFF(YEAR, CAST(T2.CREATED_AT AS DATE), CAST(GETDATE() AS DATE)) IS NULL THEN 0
		   ELSE	 DATEDIFF(YEAR, CAST(T2.CREATED_AT AS DATE), CAST(GETDATE() AS DATE)) END AS AnosDesdeCriacao
    , CASE WHEN  DATEDIFF(HOUR, CAST(T2.CREATED_AT AS DATE), CAST(GETDATE() AS DATE)) IS NULL THEN 0
	       ELSE  DATEDIFF(HOUR, CAST(T2.CREATED_AT AS DATE), CAST(GETDATE() AS DATE)) END AS HorasDesdeCriacao
    FROM Items       AS T1
    LEFT JOIN Ordens AS T2 ON T1.ID          = T2.ID 
    LEFT JOIN Produtos  T3 ON T1.PRODUCT_ID  = T3.ID
    LEFT JOIN Categoria T4 ON T3.Id_Category = T4.ID
    LEFT JOIN Clientes  T5 ON T2.CUSTOMER_ID = T5.ID
)
SELECT * FROM QUERY
WHERE ORDER_ID = 7019

"""
df_all = pd.read_sql(Query_all, conexaoDB)
df_all.head(50)

,ORDER_ID,TOTAL_PRICE,CREATED_AT,STATUS,PRODUTO,Price,Categoria,NomeCliente,email,cell_phone,Estado,Dh,DataHoje,Hora,DiasDesdeCriacao,MesesDesdeCriacao,AnosDesdeCriacao,HorasDesdeCriacao
0,7019,186.0,2019-08-28 03:36:31,entregue,Impressora,186.0,Informática,Daniel,daniel@teste.com,9 4761-6557,Rio Grande do Norte,2025-08-11 13:23:27.503,2025-08-11,13:23:27.503333,2175,72,6,52200
1,7019,15.0,2019-04-24 20:33:05,pagamento_pendente,Creme de Barbear,15.0,Cosméticos e Perfumaria,Francisco,francisco@exemplo.com,9 6754-1775,Amapá,2025-08-11 13:23:27.503,2025-08-11,13:23:27.503333,2301,76,6,55224
2,7019,15.0,2019-04-24 20:33:05,pagamento_pendente,Creme de Barbear,15.0,Cosméticos e Perfumaria,Francisco,francisco@exemplo.com,9 6754-1775,Amapá,2025-08-11 13:23:27.503,2025-08-11,13:23:27.503333,2301,76,6,55224
3,7019,156.0,NaT,None,Casinha de Boneca,39.0,Brinquedos e Games,None,None,None,None,2025-08-11 13:23:27.503,2025-08-11,13:23:27.503333,0,0,0,0
4,7019,156.0,NaT,None,Casinha de Boneca,39.0,Brinquedos e Games,None,None,None,None,2025-08-11 13:23:27.503,2025-08-11,13:23:27.503333,0,0,0,0
5,7019,328.0,NaT,None,Pebolim,82.0,Esporte e Lazer,None,None,None,None,2025-08-11 13:23:27.503,2025-08-11,13:23:27.503333,0,0,0,0
